
#Introdução — Análise de Emendas Parlamentares

Este notebook tem como objetivo responder às perguntas centrais de negócio do projeto a partir do modelo dimensional (Esquema Estrela) implementado na camada Gold.

O modelo é composto pela tabela fato Fato_Emenda, que concentra as métricas financeiras (valores empenhados, liquidados e pagos), e pelas dimensões Parlamentar, Tempo, Localidade e Orçamento, responsáveis por fornecer o contexto analítico necessário para responder às perguntas quem, quando, para onde e em quê os recursos foram destinados.

A estrutura dimensional foi projetada para permitir agregações financeiras, análises comparativas e recortes analíticos consistentes, sem dependência direta das tabelas operacionais da camada Silver.

# Respondendo as perguntas

## Qual o volume total de emendas por ano?

In [0]:
%sql
SELECT
    t.ano,
    SUM(f.valor_empenhado)  AS total_empenhado,
    SUM(f.valor_liquidado)  AS total_liquidado,
    SUM(f.valor_pago)       AS total_pago
FROM mvp.gold.fato_emenda f
JOIN mvp.gold.dim_tempo t
  ON f.sk_tempo = t.sk_tempo
GROUP BY t.ano
ORDER BY t.ano

ano,total_empenhado,total_liquidado,total_pago
2023,8812972953.67,7595962778.00,7570524374.62
2024,17254355580.74,13970439896.25,13933061634.79
2025,15697623008.56,13367523662.23,13203936037.54


O volume total de emendas parlamentares apresentou forte crescimento entre 2023 e 2024, com o orçamento praticamente dobrando nesse período. Esse aumento pode indicar maior prioridade política ou mudanças nas diretrizes de distribuição de recursos. Em 2025, observa-se uma queda no valor das emendas, sugerindo possível ajuste orçamentário ou restrições fiscais. É importante ressaltar que os dados de 2025 ainda estão em andamento, podendo sofrer alterações até o fechamento do exercício. O acompanhamento contínuo desses valores é fundamental para entender tendências e impactos das decisões parlamentares ao longo dos anos.

## Quais parlamentares movimentam mais recursos?

In [0]:
%sql
with soma_parlamentar as (
SELECT
    p.nome_parlamentar,
    SUM(f.valor_empenhado) AS total_empenhado,
    SUM(f.valor_liquidado) AS total_liquidado,
    SUM(f.valor_pago)      AS total_pago
FROM mvp.gold.fato_emenda f
JOIN mvp.gold.dim_parlamentar p
  ON f.sk_parlamentar = p.sk_parlamentar
GROUP BY p.nome_parlamentar)

select mean(total_empenhado) as media_empenhada, mean(total_liquidado) as media_liquidada, mean(total_pago) as media_pago
from soma_parlamentar

media_empenhada,media_liquidada,media_pago
86469878.970952,72326969.640745,71858223.699689


In [0]:
%sql
SELECT
    p.nome_parlamentar,
    p.sigla_partido,
    p.sigla_uf,
    SUM(f.valor_empenhado) AS total_empenhado,
    SUM(f.valor_liquidado) AS total_liquidado,
    SUM(f.valor_pago)      AS total_pago
FROM mvp.gold.fato_emenda f
JOIN mvp.gold.dim_parlamentar p
  ON f.sk_parlamentar = p.sk_parlamentar
GROUP BY p.nome_parlamentar, p.sigla_partido, sigla_uf
ORDER BY total_pago DESC
LIMIT 5

nome_parlamentar,sigla_partido,sigla_uf,total_empenhado,total_liquidado,total_pago
OLIVAL MARQUES,MDB,PA,107065970.00,107065970.00,107065970.00
CAPITAO AUGUSTO,PL,SP,107250969.00,106850969.00,106850969.00
PASTOR SARGENTO ISIDORIO,AVANTE,BA,107066677.72,106718090.08,106718090.08
OTTO ALENCAR FILHO,PSD,BA,106467221.00,106467221.00,106467221.00
MARX BELTRAO,PP,AL,107064971.00,106458744.00,106458744.00


In [0]:
%sql
SELECT
    p.nome_parlamentar,
    p.sigla_partido,
    p.sigla_uf,
    SUM(f.valor_empenhado) AS total_empenhado,
    SUM(f.valor_liquidado) AS total_liquidado,
    SUM(f.valor_pago)      AS total_pago
FROM mvp.gold.fato_emenda f
JOIN mvp.gold.dim_parlamentar p
  ON f.sk_parlamentar = p.sk_parlamentar
GROUP BY p.nome_parlamentar, p.sigla_partido, sigla_uf
ORDER BY total_pago ASC
LIMIT 5

nome_parlamentar,sigla_partido,sigla_uf,total_empenhado,total_liquidado,total_pago
RODRIGO ROLLEMBERG,PSB,DF,5390000.00,1900000.00,1900000.00
BIA KICIS,PL,DF,99731881.84,12622785.89,12593684.96
MARCELO CRIVELLA,REPUBLICANOS,RJ,60035611.70,14335595.40,14335595.40
PASTOR HENRIQUE VIEIRA,PSOL,RJ,67773186.54,17584605.25,14966356.05
TARCISIO MOTTA,PSOL,RJ,63273265.87,18718173.24,16029363.36


Os parlamentares que movimentam os maiores volumes de recursos por meio de emendas não são, necessariamente, os nomes mais midiáticos frequentemente presentes em veículos como CNN, Globo ou Folha. Observa-se que esses parlamentares de maior atuação financeira estão utilizando valores próximos ao limite permitido para emendas parlamentares, evidenciando uma atuação intensa, porém menos visível na mídia tradicional. Vale ressaltar que o uso da cota de emendas parlamentares não implica necessariamente em desvio de verba, e essas verbas podem estar sendo bem destinadas conforme as necessidades das localidades beneficiadas.


Chama atenção o fato de alguns parlamentares apresentarem valores baixos pagos em relação ao total empenhado nas emendas parlamentares. Por exemplo, Marcello Crivella empenhou cerca de 60 milhões, mas apenas 14 milhões foram efetivamente pagos. Essa diferença pode ocorrer por diversos motivos, como bloqueios orçamentários, pendências na execução dos projetos, falta de documentação, ou restrições fiscais que impedem a liberação total dos recursos. É importante analisar cada caso para entender os fatores que influenciam o pagamento das emendas.

## Quais partidos destinam mais verbas por área (saúde, educação etc.)?

In [0]:
%sql
select *
from mvp.gold.dim_orcamento

sk_orcamento,funcao
0,Múltiplo
1,Organização agrária
2,Desporto e lazer
3,Ciência e Tecnologia
4,Encargos especiais
5,Assistência social
6,Comércio e serviços
7,Saneamento
8,Relações exteriores
9,Essencial à justiça


In [0]:
%sql
WITH gasto_partido_funcao AS (
    SELECT
        p.sigla_partido,
        o.funcao,
        SUM(f.valor_pago) AS total_pago_funcao
    FROM mvp.gold.fato_emenda f
    JOIN mvp.gold.dim_parlamentar p
      ON f.sk_parlamentar = p.sk_parlamentar
    JOIN mvp.gold.dim_orcamento o
      ON f.sk_orcamento = o.sk_orcamento
    WHERE o.funcao = 'Saúde'
    GROUP BY
        p.sigla_partido,
        o.funcao
),
gasto_total_partido AS (
    SELECT
        p.sigla_partido,
        SUM(f.valor_pago) AS total_pago_partido
    FROM mvp.gold.fato_emenda f
    JOIN mvp.gold.dim_parlamentar p
      ON f.sk_parlamentar = p.sk_parlamentar
    GROUP BY
        p.sigla_partido
)

SELECT
    gpf.sigla_partido,
    gpf.funcao,
    gpf.total_pago_funcao,
    gtp.total_pago_partido,
    gpf.total_pago_funcao / gtp.total_pago_partido AS percentual_funcao_no_partido
FROM gasto_partido_funcao gpf
JOIN gasto_total_partido gtp
  ON gpf.sigla_partido = gtp.sigla_partido
ORDER BY percentual_funcao_no_partido DESC

sigla_partido,funcao,total_pago_funcao,total_pago_partido,percentual_funcao_no_partido
NOVO,Saúde,162669696.00,237903012.23,0.6837647597
PSOL,Saúde,235182533.83,364519829.19,0.6451844728
REDE,Saúde,38496490.00,63347370.33,0.6077046261
PSD,Saúde,1991493421.00,3457427511.79,0.5760043889
UNIÃO,Saúde,2195011086.00,3815439452.62,0.5752970564
PT,Saúde,2482102878.71,4350156107.09,0.5705778868
PP,Saúde,2004921904.00,3553440729.79,0.5642198805
PODE,Saúde,524909889.00,930940293.20,0.5638491457
PL,Saúde,3513969806.00,6234154386.41,0.5636642258
REPUBLICANOS,Saúde,1687858611.00,3013281911.45,0.5601396287


A maioria dos partidos, independentemente de orientação política, direciona grande parte das emendas parlamentares para a área da Saúde, em conformidade com a legislação que exige a destinação de pelo menos 50% dos recursos para esse setor. [Fonte](https://www.gov.br/igualdaderacial/pt-br/acesso-a-informacao/assessoria-parlamentar/emendas-parlamentares/ManualdeEmendasParlamentaresdigital.pdf?utm_source=chatgpt.com) <br>
Destaca-se, porém, que o MDB apresenta um percentual abaixo do mínimo exigido, o que merece atenção.


In [0]:
%sql
WITH gasto_partido_funcao AS (
    SELECT
        p.sigla_partido,
        o.funcao,
        SUM(f.valor_pago) AS total_pago_funcao
    FROM mvp.gold.fato_emenda f
    JOIN mvp.gold.dim_parlamentar p
      ON f.sk_parlamentar = p.sk_parlamentar
    JOIN mvp.gold.dim_orcamento o
      ON f.sk_orcamento = o.sk_orcamento
    WHERE o.funcao = 'Educação'
    GROUP BY
        p.sigla_partido,
        o.funcao
),
gasto_total_partido AS (
    SELECT
        p.sigla_partido,
        SUM(f.valor_pago) AS total_pago_partido
    FROM mvp.gold.fato_emenda f
    JOIN mvp.gold.dim_parlamentar p
      ON f.sk_parlamentar = p.sk_parlamentar
    GROUP BY
        p.sigla_partido
)

SELECT
    gpf.sigla_partido,
    gpf.funcao,
    gpf.total_pago_funcao,
    gtp.total_pago_partido,
    gpf.total_pago_funcao / gtp.total_pago_partido AS percentual_funcao_no_partido
FROM gasto_partido_funcao gpf
JOIN gasto_total_partido gtp
  ON gpf.sigla_partido = gtp.sigla_partido
ORDER BY percentual_funcao_no_partido DESC

sigla_partido,funcao,total_pago_funcao,total_pago_partido,percentual_funcao_no_partido
PSOL,Educação,59944834.93,364519829.19,0.1644487628
SOLIDARIEDADE,Educação,18161423.64,433866746.41,0.0418594506
PCdoB,Educação,9127721.10,532994750.16,0.0171253490
PT,Educação,67689333.99,4350156107.09,0.0155602080
CIDADANIA,Educação,2315290.32,267461703.73,0.0086565302
PSD,Educação,23879506.80,3457427511.79,0.0069067267
UNIÃO,Educação,18193907.89,3815439452.62,0.0047684960
PDT,Educação,4361350.19,1131564118.58,0.0038542670
MDB,Educação,10159597.99,2990281501.55,0.0033975390
PL,Educação,18822826.35,6234154386.41,0.0030193071


Os partidos de esquerda se destacam quando analisamos os gastos com educação, apresentando percentuais superiores aos demais grupos políticos. No entanto, o volume destinado à área ainda é considerado baixo no contexto geral das emendas parlamentares, com exceção do PSol, que alocou 16% do seu orçamento para educação — valor significativamente acima da média dos demais partidos.

In [0]:
%sql
WITH gasto_partido_funcao AS (
    SELECT
        p.sigla_partido,
        o.funcao,
        SUM(f.valor_pago) AS total_pago_funcao
    FROM mvp.gold.fato_emenda f
    JOIN mvp.gold.dim_parlamentar p
      ON f.sk_parlamentar = p.sk_parlamentar
    JOIN mvp.gold.dim_orcamento o
      ON f.sk_orcamento = o.sk_orcamento
    WHERE o.funcao = 'Segurança pública'
    GROUP BY
        p.sigla_partido,
        o.funcao
),
gasto_total_partido AS (
    SELECT
        p.sigla_partido,
        SUM(f.valor_pago) AS total_pago_partido
    FROM mvp.gold.fato_emenda f
    JOIN mvp.gold.dim_parlamentar p
      ON f.sk_parlamentar = p.sk_parlamentar
    GROUP BY
        p.sigla_partido
)

SELECT
    gpf.sigla_partido,
    gpf.funcao,
    gpf.total_pago_funcao,
    gtp.total_pago_partido,
    gpf.total_pago_funcao / gtp.total_pago_partido AS percentual_funcao_no_partido
FROM gasto_partido_funcao gpf
JOIN gasto_total_partido gtp
  ON gpf.sigla_partido = gtp.sigla_partido
ORDER BY percentual_funcao_no_partido DESC

sigla_partido,funcao,total_pago_funcao,total_pago_partido,percentual_funcao_no_partido
NOVO,Segurança pública,1394400.00,237903012.23,0.0058612120
PSOL,Segurança pública,2000269.06,364519829.19,0.0054874081
PT,Segurança pública,9250463.23,4350156107.09,0.0021264670
PL,Segurança pública,10120318.90,6234154386.41,0.0016233667
PP,Segurança pública,4632488.05,3553440729.79,0.0013036627
PSDB,Segurança pública,1207844.54,941451293.99,0.0012829602
REPUBLICANOS,Segurança pública,3493000.00,3013281911.45,0.0011592012
PSB,Segurança pública,900309.09,1020739259.51,0.0008820167
PDT,Segurança pública,946600.00,1131564118.58,0.0008365412
UNIÃO,Segurança pública,3007473.90,3815439452.62,0.0007882379


Novamente, quando olhamos para os gastos em segurança pública, eles são baixos, e vemos partidos de direita e esquerda com gastos próximos. Isso sugere que, independentemente da orientação política, a destinação de recursos para essa área não é prioridade nas emendas parlamentares, refletindo talvez uma preferência por setores como saúde e educação.

## Para onde (UF/município) cada parlamentar direciona sua verba?

In [0]:
%sql
SELECT
    p.nome_parlamentar,
    l.uf_destino,
    l.municipio_destino,
    SUM(f.valor_pago) AS total_pago
FROM mvp.gold.fato_emenda f
JOIN mvp.gold.dim_parlamentar p
  ON f.sk_parlamentar = p.sk_parlamentar
JOIN mvp.gold.dim_localidade l
  ON f.sk_localidade = l.sk_localidade
WHERE p.nome_parlamentar = 'TABATA AMARAL'
GROUP BY
    p.nome_parlamentar,
    l.uf_destino,
    l.municipio_destino
ORDER BY
    p.nome_parlamentar,
    total_pago DESC


nome_parlamentar,uf_destino,municipio_destino,total_pago
TABATA AMARAL,null,null,28047776.51
TABATA AMARAL,SP,null,5364290.27
TABATA AMARAL,SP,SÃO VICENTE,200000.00


Com essa consulta é possível visualizar as localidades para as quais as emendas parlamentares estão sendo destinadas. Observa-se que a Tabata direciona boa parte de seu orçamento sem especificar um estado. Quando há indicação de estado de destino, este corresponde ao estado pelo qual foi eleita.

## Qual a razão entre verba empenhada vs. paga? Há diferenças significativas?

In [0]:
%sql
SELECT
    t.ano,
    SUM(f.valor_pago) / SUM(f.valor_empenhado) AS taxa_execucao
FROM mvp.gold.fato_emenda f
JOIN mvp.gold.dim_tempo t
  ON f.sk_tempo = t.sk_tempo
WHERE f.valor_empenhado > 0
GROUP BY t.ano
ORDER BY t.ano

ano,taxa_execucao
2023,0.8590204934
2024,0.8075098238
2025,0.8411423838


Podemos ver que a taxa de execução do orçamento é, em média, por volta de 80%. Em 2024, esse índice ficou abaixo da média, atingindo 80,7%. Já nos anos de 2023 e 2025, a taxa de execução se aproximou dos 85%, indicando melhor desempenho na liberação dos recursos empenhados nesses períodos.

## Quais áreas recebem mais recursos ao longo do tempo?

In [0]:
%sql
SELECT
    t.ano,
    o.funcao,
    SUM(f.valor_pago) AS total_pago
FROM mvp.gold.fato_emenda f
JOIN mvp.gold.dim_tempo t
  ON f.sk_tempo = t.sk_tempo
JOIN mvp.gold.dim_orcamento o
  ON f.sk_orcamento = o.sk_orcamento
GROUP BY
    t.ano,
    o.funcao
ORDER BY
    t.ano,
    total_pago DESC

ano,funcao,total_pago
2023,Saúde,4068100589.28
2023,Encargos especiais,3070492921.00
2023,Assistência social,162398017.54
2023,Educação,69597575.30
2023,Desporto e lazer,59115619.04
2023,Agricultura,32331810.39
2023,Defesa nacional,31026528.00
2023,Múltiplo,28344693.52
2023,Cultura,22643856.00
2023,Direitos da cidadania,7783082.32



Saúde, encargos especiais e assistência social sempre são os maiores gastos de emendas parlamentares.

# Autoavaliação

**Autoavaliação do Projeto**

De forma geral, os objetivos propostos neste notebook foram plenamente atendidos.
A modelagem dimensional construída permite analisar as emendas parlamentares sob diferentes perspectivas analíticas, incluindo:

- Quem destinou os recursos (parlamentar e partido);

- Quando os recursos foram alocados (dimensão temporal);

- Para onde os recursos foram direcionados (localidade);

- Em que área os recursos foram aplicados (função orçamentária);

- Quais valores foram empenhados, liquidados e efetivamente pagos.

A estrutura em Esquema Estrela, com uma tabela fato central e dimensões bem definidas, mostrou-se adequada para responder às perguntas de negócio estabelecidas previamente, garantindo simplicidade, desempenho analítico e clareza conceitual.

Além disso, destaca-se que:

- Todas as tabelas criadas na camada Gold possuem documentação, incluindo comentários em nível de tabela e coluna;

- Foi realizado um desenho explícito da modelagem dimensional, facilitando a compreensão do modelo, suas chaves e relacionamentos, bem como o uso por outros analistas e consumidores de dados.

⚠️ **Pontos Não Cumpridos Completamente / Limitações Identificadas**

Apesar dos resultados positivos, algumas limitações foram identificadas ao longo do desenvolvimento:

1. Aquisição dos dados
- As emendas parlamentares precisaram ser baixadas diretamente do site oficial, pois não foi possível estabelecer um acesso efetivo via API.
Isso dificulta a automação completa do pipeline, tornando o processo mais dependente de intervenções manuais.

2. Vinculação entre emendas e parlamentares
  - A associação entre as tabelas de emendas parlamentares e deputados não é perfeita. Existem parlamentares em atividade que não puderam ser vinculados corretamente, principalmente devido a problemas de padronização de nomes, como variações ortográficas e inconsistências na fonte de dados.

3. Qualidade da informação de localidade
  - A coluna de localidade presente nas emendas não possui um padrão único, o que limita análises geográficas mais refinadas.
  Em diferentes registros, a localidade pode representar:
    - Um estado (UF),

    - Uma região,

    - Um município,

    - Ou múltiplas localidades simultaneamente.

  - Essa variabilidade reduz a precisão da dimensão de localidade e aponta para a necessidade de um tratamento semântico e geográfico mais avançado.

4. Possíveis melhorias futuras
Como evolução natural do projeto, seriam desejáveis:

- Normalização geográfica mais robusta (ex.: uso de códigos IBGE);

- Estratégias de entity resolution para parlamentares;

- Automatização da ingestão de dados a partir de fontes oficiais estruturadas.